# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:

stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [8]:
symbol = 'AAPL'

# basic stats endpoint
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/stats/?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2302193213347,
 'week52high': 185.76,
 'week52low': 132.13,
 'week52highSplitAdjustOnly': 188.45,
 'week52lowSplitAdjustOnly': 133.49,
 'week52change': 0.04399610225321419,
 'sharesOutstanding': 16252257943,
 'float': 0,
 'avg10Volume': 94040215,
 'avg30Volume': 89404598,
 'day200MovingAvg': 167.39,
 'day50MovingAvg': 145.63,
 'employees': 147016,
 'ttmEPS': 6.26,
 'ttmDividendRate': 0.9124818907465978,
 'dividendYield': 0.006610580706531937,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-30',
 'nextEarningsDate': '2022-07-19',
 'peRatio': 22.45963727322552,
 'beta': 1.2837162318124609,
 'maxChangePercent': 52.9182904092264,
 'year5ChangePercent': 3.0910322834978152,
 'year2ChangePercent': 0.534975300912217,
 'year1ChangePercent': 0.04096450723126,
 'ytdChangePercent': -0.21979170519110486,
 'month6ChangePercent': -0.2176333652944921,
 'month3ChangePercent': -0.19362901987636766,
 'month1ChangePercent': -0.03503012830633728,
 'day30Chang

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [11]:
data['year1ChangePercent']

0.04096450723126

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [15]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [68]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,138.24,-0.327227,N/A
1,AOS,58.27,-0.198024,N/A
2,ABT,109.53,-0.028047,N/A
3,ABBV,158.33,0.372340,N/A
4,ABMD,259.45,-0.233060,N/A
...,...,...,...,...
497,YUM,114.87,-0.023806,N/A
498,ZBRA,311.78,-0.428944,N/A
499,ZBH,110.43,-0.352035,N/A
500,ZION,55.47,-0.043473,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [69]:
# inplace = True changes function from returning a sorted dataframe vs modifying it
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,56.02,1.032797,N/A
1,OXY,59.89,0.829024,N/A
2,MCK,336.67,0.695975,N/A
3,APA,37.16,0.671707,N/A
4,MRO,23.03,0.667851,N/A
5,CF,84.75,0.632066,N/A
6,CTRA,27.11,0.600230,N/A
7,DLTR,157.24,0.551218,N/A
8,VRTX,302.31,0.525561,N/A
9,COP,93.97,0.508950,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [70]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)

    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

10000000


In [76]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe

/Users/kyleshum/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,56.02,1.032797,3570
1,OXY,59.89,0.829024,3339
2,MCK,336.67,0.695975,594
3,APA,37.16,0.671707,5382
4,MRO,23.03,0.667851,8684
5,CF,84.75,0.632066,2359
6,CTRA,27.11,0.600230,7377
7,DLTR,157.24,0.551218,1271
8,VRTX,302.31,0.525561,661
9,COP,93.97,0.508950,2128


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [138]:
hqm_columns = [
    'Ticker', 
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                    symbol, 
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
            ],
            index = hqm_columns),
            ignore_index=True
        ) 

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,141,N/A,-0.330146,N/A,-0.261232,N/A,-0.121146,N/A,-0.099806,N/A,N/A
1,AOS,55.95,N/A,-0.197845,N/A,-0.346148,N/A,-0.171522,N/A,-0.074073,N/A,N/A
2,ABT,111.67,N/A,-0.028089,N/A,-0.234934,N/A,-0.091635,N/A,-0.077419,N/A,N/A
3,ABBV,157.68,N/A,0.378450,N/A,0.149105,N/A,-0.048993,N/A,0.009818,N/A,N/A
4,ABMD,264.39,N/A,-0.228107,N/A,-0.298488,N/A,-0.202769,N/A,-0.002456,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,118.89,N/A,-0.023531,N/A,-0.163959,N/A,-0.027652,N/A,-0.008251,N/A,N/A
498,ZBRA,323.72,N/A,-0.424472,N/A,-0.495036,N/A,-0.291754,N/A,-0.116352,N/A,N/A
499,ZBH,108.93,N/A,-0.365404,N/A,-0.165802,N/A,-0.120307,N/A,-0.123438,N/A,N/A
500,ZION,53.57,N/A,-0.045115,N/A,-0.186416,N/A,-0.249585,N/A,-0.066277,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [139]:
time_periods = [

                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col]) / 100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,141,N/A,-0.330146,0.145418,-0.261232,0.316733,-0.121146,0.539841,-0.099806,0.280876,N/A
1,AOS,55.95,N/A,-0.197845,0.316733,-0.346148,0.143426,-0.171522,0.376494,-0.074073,0.424303,N/A
2,ABT,111.67,N/A,-0.028089,0.59761,-0.234934,0.384462,-0.091635,0.645418,-0.077419,0.410359,N/A
3,ABBV,157.68,N/A,0.378450,0.964143,0.149105,0.932271,-0.048993,0.749004,0.009818,0.856574,N/A
4,ABMD,264.39,N/A,-0.228107,0.268924,-0.298488,0.241036,-0.202769,0.276892,-0.002456,0.796813,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,118.89,N/A,-0.023531,0.609562,-0.163959,0.523904,-0.027652,0.804781,-0.008251,0.770916,N/A
498,ZBRA,323.72,N/A,-0.424472,0.071713,-0.495036,0.031873,-0.291754,0.093625,-0.116352,0.201195,N/A
499,ZBH,108.93,N/A,-0.365404,0.111554,-0.165802,0.51992,-0.120307,0.543825,-0.123438,0.187251,N/A
500,ZION,53.57,N/A,-0.045115,0.581673,-0.186416,0.48008,-0.249585,0.159363,-0.066277,0.47012,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [140]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])

    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,141,N/A,-0.330146,0.145418,-0.261232,0.316733,-0.121146,0.539841,-0.099806,0.280876,0.320717
1,AOS,55.95,N/A,-0.197845,0.316733,-0.346148,0.143426,-0.171522,0.376494,-0.074073,0.424303,0.315239
2,ABT,111.67,N/A,-0.028089,0.59761,-0.234934,0.384462,-0.091635,0.645418,-0.077419,0.410359,0.509462
3,ABBV,157.68,N/A,0.378450,0.964143,0.149105,0.932271,-0.048993,0.749004,0.009818,0.856574,0.875498
4,ABMD,264.39,N/A,-0.228107,0.268924,-0.298488,0.241036,-0.202769,0.276892,-0.002456,0.796813,0.395916
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,118.89,N/A,-0.023531,0.609562,-0.163959,0.523904,-0.027652,0.804781,-0.008251,0.770916,0.677291
498,ZBRA,323.72,N/A,-0.424472,0.071713,-0.495036,0.031873,-0.291754,0.093625,-0.116352,0.201195,0.099602
499,ZBH,108.93,N/A,-0.365404,0.111554,-0.165802,0.51992,-0.120307,0.543825,-0.123438,0.187251,0.340637
500,ZION,53.57,N/A,-0.045115,0.581673,-0.186416,0.48008,-0.249585,0.159363,-0.066277,0.47012,0.422809


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [141]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)  # drop = True removes old index
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VRTX,296.48,N/A,0.521700,0.982072,0.269137,0.972112,0.147955,0.994024,0.059103,0.954183,0.975598
1,LLY,327.46,N/A,0.468495,0.974104,0.153937,0.936255,0.107028,0.982072,0.033634,0.910359,0.950697
2,AZO,2260.09,N/A,0.466600,0.972112,0.048977,0.846614,0.071346,0.958167,0.173597,0.994024,0.942729
3,DLTR,162.19,N/A,0.542801,0.986056,0.135858,0.924303,0.002392,0.860558,0.204489,0.998008,0.942231
4,HSY,227.72,N/A,0.291093,0.942231,0.160186,0.938247,0.051573,0.942231,0.043842,0.934263,0.939243
5,DG,253.5,N/A,0.154078,0.87251,0.107698,0.896414,0.105311,0.98008,0.261561,1.0,0.937251
6,BMY,81.47,N/A,0.236062,0.926295,0.273272,0.974104,0.098671,0.976096,0.012992,0.864542,0.935259
7,ENPH,206.98,N/A,0.184359,0.900398,0.066449,0.860558,0.055196,0.948207,0.187439,0.996016,0.926295
8,MRK,97.03,N/A,0.270440,0.936255,0.236989,0.968127,0.164911,0.996016,-0.011841,0.747012,0.911853
9,NOC,464.6,N/A,0.254070,0.930279,0.207225,0.960159,0.025715,0.90239,0.002590,0.832669,0.906375


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [142]:
portfolio_input()

In [148]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

/Users/kyleshum/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VRTX,296.48,674,0.521700,0.982072,0.269137,0.972112,0.147955,0.994024,0.059103,0.954183,0.975598
1,LLY,327.46,610,0.468495,0.974104,0.153937,0.936255,0.107028,0.982072,0.033634,0.910359,0.950697
2,AZO,2260.09,88,0.466600,0.972112,0.048977,0.846614,0.071346,0.958167,0.173597,0.994024,0.942729
3,DLTR,162.19,1233,0.542801,0.986056,0.135858,0.924303,0.002392,0.860558,0.204489,0.998008,0.942231
4,HSY,227.72,878,0.291093,0.942231,0.160186,0.938247,0.051573,0.942231,0.043842,0.934263,0.939243
5,DG,253.5,788,0.154078,0.87251,0.107698,0.896414,0.105311,0.98008,0.261561,1.0,0.937251
6,BMY,81.47,2454,0.236062,0.926295,0.273272,0.974104,0.098671,0.976096,0.012992,0.864542,0.935259
7,ENPH,206.98,966,0.184359,0.900398,0.066449,0.860558,0.055196,0.948207,0.187439,0.996016,0.926295
8,MRK,97.03,2061,0.270440,0.936255,0.236989,0.968127,0.164911,0.996016,-0.011841,0.747012,0.911853
9,NOC,464.6,430,0.254070,0.930279,0.207225,0.960159,0.025715,0.90239,0.002590,0.832669,0.906375


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [149]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [150]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [151]:
column_formats = {
    'A':['Ticker', string_template], 
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template ],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 25, column_formats[column][1])
                                               # replacing first cell, name of col, format of column
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [152]:
writer.save()